# Knapsack Problem solved using genetic algorithms

In [2]:
# Imports 
import numpy as np
import random

from datetime import datetime

In [18]:
# Parameters
n_objects = 20
max_weight = 3

n_population = 100

mutation_rate = 0.3

In [23]:
# Generating a list of coordenades representing each city
weight_value = [[x,y] for x,y in zip(np.random.randint(0,10,n_objects)/10,np.random.randint(0,100,n_objects))]
object_list = np.array(['Water', 'Food', 'Pants', 'Socks', 'Boots', 'Shirts', 'Coat', 'Blanket', 'Laptop', 'TV', 'Cellphone', 'Book', 'Gloves', 'Towel', 'Sunscream', 'Glasses', 'Fork', 'Knife', 'Matches', 'Chair'])
objects_dict = { x:y for x,y in zip(object_list,weight_value)}

def get_current_weight_value(objects_list, objects_dict):
    weight = 0
    value = 0
    for o in objects_list:
        o = objects_dict[o]
        weight += o[0]
        value += o[1]
    return weight, value
        

def fit_in_knapsack(objects_list, max_weight, objects_dict):
    r = []
    for o in objects_list:
        r.append(o)
        weight, value = get_current_weight_value(r, objects_dict)
        if weight > max_weight:
            r.pop()
            return r
    return r
    
objects_dict

{'Water': [0.9, 56],
 'Food': [0.7, 22],
 'Pants': [0.4, 72],
 'Socks': [0.4, 94],
 'Boots': [0.0, 73],
 'Shirts': [0.2, 75],
 'Coat': [0.0, 67],
 'Blanket': [0.7, 79],
 'Laptop': [0.9, 19],
 'TV': [0.2, 75],
 'Cellphone': [0.0, 8],
 'Book': [0.0, 74],
 'Gloves': [0.4, 32],
 'Towel': [0.1, 40],
 'Sunscream': [0.3, 59],
 'Glasses': [0.3, 60],
 'Fork': [0.0, 47],
 'Knife': [0.1, 65],
 'Matches': [0.3, 42],
 'Chair': [0.9, 33]}

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [24]:
# First step: Create the first population set
def genesis(object_list, n_population, max_weight, objects_dict):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = object_list[np.random.choice(list(range(n_objects)), n_objects, replace=False)]
        sol_i = fit_in_knapsack(sol_i, max_weight, objects_dict)
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(object_list, n_population, max_weight, objects_dict)
population_set

<ipython-input-24-2ef81bb1ccef>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(population_set)


array([list(['Fork', 'Water', 'Matches', 'Gloves', 'TV', 'Book', 'Laptop']),
       list(['Water', 'Pants', 'Cellphone', 'Laptop']),
       list(['Food', 'Cellphone', 'Book', 'Gloves', 'Coat', 'Pants', 'Knife', 'Socks', 'Laptop']),
       list(['Blanket', 'Cellphone', 'Book', 'Pants', 'Food', 'Matches', 'Socks', 'Boots', 'Knife']),
       list(['Laptop', 'Chair', 'Fork', 'Boots', 'Blanket', 'Shirts']),
       list(['Matches', 'Shirts', 'Book', 'Glasses', 'Socks', 'Laptop', 'TV', 'Fork', 'Blanket']),
       list(['Blanket', 'Gloves', 'TV', 'Laptop', 'Book', 'Socks']),
       list(['Water', 'Pants', 'Food', 'Laptop', 'Towel', 'Coat', 'Fork']),
       list(['Shirts', 'Knife', 'Book', 'Fork', 'Pants', 'Glasses', 'Food', 'TV', 'Cellphone', 'Matches']),
       list(['Food', 'Gloves', 'Blanket', 'TV', 'Boots', 'Fork', 'Chair', 'Coat', 'Towel']),
       list(['Gloves', 'TV', 'Chair', 'Boots', 'Glasses', 'Fork', 'Blanket', 'Towel', 'Cellphone', 'Knife', 'Matches']),
       list(['Shirts', 'Food

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [26]:
def get_all_fitnes(population_set, objects_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        _, fitnes_list[i] = get_current_weight_value(population_set[i], objects_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([292., 191., 442., 446., 182., 312., 254., 206., 429., 314., 449.,
       160., 304., 350., 530., 474., 281., 369., 409., 419., 495., 341.,
       465., 372., 313., 333., 309., 276., 232., 393., 164., 478., 361.,
       186., 378., 393., 479., 333., 365., 296., 313., 206., 311., 441.,
       562., 310., 558., 239., 440., 332., 379., 414., 372., 253., 337.,
       521., 191., 316., 416., 518., 350., 382., 252., 163., 307., 323.,
       242., 257., 314., 386., 347., 445., 351., 263., 351., 435., 444.,
       410., 329., 288., 397., 253., 267., 276., 460., 429., 472., 305.,
       418., 393., 378., 326., 342., 326., 303., 592., 218., 257., 399.,
       368.])

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [27]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

['Gloves',
 'Socks',
 'Pants',
 'Blanket',
 'Fork',
 'Matches',
 'Cellphone',
 'Knife',
 'TV']

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [42]:
def mate_progenitors(prog_a, prog_b, max_weight, objects_dict):
    offspring = []

    for i in zip(prog_a, prog_b):
        offspring.extend(i)
        offspring = list(dict.fromkeys(offspring)) #Removing duplicates
        offspring = fit_in_knapsack(offspring, max_weight, objects_dict)

    return offspring
            
    
    
def mate_population(progenitor_list, max_weight, objects_dict):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b, max_weight, objects_dict)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list, max_weight, objects_dict)
new_population_set[0]

['Socks',
 'TV',
 'Matches',
 'Towel',
 'Sunscream',
 'Book',
 'Knife',
 'Food',
 'Coat',
 'Pants']

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [45]:
def mutate_offspring(offspring, max_weight, object_list, objects_dict):
    for mutation_number in range(int(len(offspring)*mutation_rate)):
    
        a = np.random.randint(0,len(object_list))
        b = np.random.randint(0,len(offspring))
        
        offspring.insert(b, object_list[a])
        
    offspring = fit_in_knapsack(offspring, max_weight, objects_dict)

    return offspring
    
    
def mutate_population(new_population_set, max_weight, object_list, objects_dict):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring, max_weight, object_list, objects_dict))
    return mutated_pop

mutated_pop = mutate_population(new_population_set, max_weight,object_list, objects_dict)
mutated_pop[0]

['Socks',
 'TV',
 'Chair',
 'Book',
 'Matches',
 'Towel',
 'Sunscream',
 'Book',
 'Coat',
 'Knife',
 'Food',
 'Coat']

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [47]:
best_solution = [-1,-np.inf,np.array([])]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.max() > best_solution[0]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.max()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.max() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list, max_weight, objects_dict)
    
    mutated_pop = mutate_population(new_population_set, max_weight,object_list, objects_dict)

0 142.0 415.06 31/01/21 12:05


<ipython-input-47-b04ed29c054d>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  best_solution[2] = np.array(mutated_pop)[fitnes_list.max() == fitnes_list]


100 162.0 385.01 31/01/21 12:05
200 108.0 366.88 31/01/21 12:06
300 107.0 376.34 31/01/21 12:06
400 173.0 380.67 31/01/21 12:06
500 133.0 353.73 31/01/21 12:06
600 159.0 397.97 31/01/21 12:06
700 138.0 358.58 31/01/21 12:06
800 97.0 378.15 31/01/21 12:06
900 164.0 350.29 31/01/21 12:06
1000 150.0 394.45 31/01/21 12:06
1100 84.0 375.76 31/01/21 12:06
1200 96.0 365.41 31/01/21 12:06
1300 142.0 383.28 31/01/21 12:06
1400 122.0 357.45 31/01/21 12:06
1500 144.0 355.38 31/01/21 12:06
1600 74.0 354.52 31/01/21 12:06
1700 176.0 371.07 31/01/21 12:06
1800 96.0 379.2 31/01/21 12:06
1900 129.0 384.92 31/01/21 12:06
2000 130.0 368.94 31/01/21 12:06
2100 127.0 370.75 31/01/21 12:06
2200 160.0 355.67 31/01/21 12:06
2300 181.0 378.77 31/01/21 12:06
2400 153.0 388.26 31/01/21 12:06
2500 116.0 377.21 31/01/21 12:06
2600 142.0 358.79 31/01/21 12:06
2700 145.0 385.91 31/01/21 12:06
2800 153.0 356.38 31/01/21 12:06
2900 194.0 366.09 31/01/21 12:06
3000 161.0 376.76 31/01/21 12:06
3100 179.0 362.46 31/01/2

KeyboardInterrupt: 

In [53]:
get_current_weight_value(best_solution[2][0], objects_dict)

(2.9, 868)